[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/liuguangw/webui_colab/blob/master/webui.ipynb)

In [ ]:
# @title 1.安装Linux基本环境
!apt -y update
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

%pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!apt -y install aria2
%pip install xformers==0.0.16
%pip install triton==2.0.0


In [ ]:
# @title 2.挂载云端硬盘

# @markdown 需要授予权限

# @markdown `models`和`outputs`目录的文件会保存到云端硬盘,
# @markdown `settings` 目录保存配置文件

%cd /content

repository_path = "/content/stable-diffusion-webui"
!rm -rf {repository_path}
repository_url = "https://github.com/AUTOMATIC1111/stable-diffusion-webui"  # @param {type: "string"}
webui_hash = "22bcc7be428c94e9408f589966c2040187245d81"  # @param {type: "string"}
!git clone {repository_url}
%cd {repository_path}
!git checkout {webui_hash}

# @markdown 云端文件夹目录
data_dir_name = "sd_webui_colab"  # @param {type:"string"}
data_dir = f"/content/drive/MyDrive/{data_dir_name}"

import os

drive_path = "/content/drive"
from google.colab import drive

try:
    drive.mount(drive_path, force_remount=True)
    os.makedirs(data_dir, exist_ok=True)
except:
    print('Mount Error')
    raise

models_path = f"{data_dir}/models"
outputs_path = f"{data_dir}/outputs"
settings_path = f"{data_dir}/settings"


if not os.path.exists(models_path):
  !cp -r {repository_path}/models {data_dir}/

if not os.path.exists(outputs_path):
  !mkdir -p {outputs_path}

if not os.path.exists(settings_path):
  !mkdir -p {settings_path}

!rm -rf {repository_path}/models
!rm -rf {repository_path}/outputs
!ln -s {models_path} {repository_path}/
!ln -s {outputs_path} {repository_path}/

#配置文件路径定义
os.environ['STYLES_FILE'] = f"{settings_path}/styles.csv"
os.environ['UI_CONFIG_FILE'] = f"{settings_path}/ui-config.json"
os.environ['UI_SETTINGS_FILE'] = f"{settings_path}/config.json"

!rm -rf {repository_path}/venv

In [ ]:
# @title 3.安装扩展(可选操作)
# @markdown 可选操作
%cd {repository_path}/extensions
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
!git clone https://github.com/VinsonLaro/stable-diffusion-webui-chinese.git
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git
!git clone https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper.git

In [ ]:
# @title 4.启动webui
import os
%cd {repository_path}
#! ls -al {data_dir}/models
# @markdown 启动参数
up_args = "--xformers --enable-insecure-extension-access --share" # @param {type:"string"}

# @markdown 主题
darkMode = True # @param {type:"boolean"}
if darkMode:
  up_args = f"{up_args} --theme dark"


# @markdown 是否使用ngrok
ngrok = False # @param {type:"boolean"}
ngrok_token = "" # @param {type:"string"}

if ngrok:
  up_args = f"{up_args} --ngrok {ngrok_token}"

up_args = f"{up_args} --ui-settings-file {settings_path}/config.json"
up_args = f"{up_args} --ui-config-file {settings_path}/ui-config.json"
up_args = f"{up_args} --styles-file {settings_path}/styles.csv"

!python launch.py {up_args}